<a href="https://colab.research.google.com/github/wlsisl/ESAA_/blob/main/ch09%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 08 파이선 추천 시스템 패키지 - Surprise

### Surprise 패키지 소개

- 장점
 
 * 다양한 추천 알고리즘을 쉽게 적용해 추천 시스템 구축 가능
 * surprise의 핵심 api는 사이킷런의 핵심 api와 유사한 api명으로 작성됨. 

In [2]:
! pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095474 sha256=a74e9547f399a2ddc8e81371f5d506e72bed396a5d9ef4e02deb077a39657426
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


### Surprise를 이용한 추천 시스템 구축

추천 데이터를 학습용 테스트용 데이터 세트로 분리한 뒤 SVD행렬 분해를 통한 잠재 요인 협업 필터링을 수행.

In [3]:
#관련 모듈 import
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

* 데이터 로딩은 Dataset 클래스를 이용해서만 가능.
* 주요 데이터가 로우 레벨 형태로 돼있는 포맷의 데이터만 처리

In [5]:
data=Dataset.load_builtin('ml-100k')
#수행 시마다 동일하게 데이터를 분할하기 위해 random_state부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [6]:
# svd로 잠재 요인 협업 필터링 수행
algo=SVD() #알고리즘객체 생성
algo.fit(trainset)

test() : 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드 ( 추천 데이터 세트를 만들어줌 )

predict() : 개별 사용자와 영화에 대한 추천 평점을 반환

In [7]:
#테스트 데이터 세트 전체에 대해 추천 영화 평점 데이터 생성 후 추출
predictions=algo.test(testset)
print('predictions type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

predictions type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6682630479069185, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8825855670016187, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.065480493123735, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4024805902022304, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.0408225614384055, details={'was_impossible': False})]

개별 사용자 아이디(uid), 영화 아이디(iid), 실제 평점(r_ui), 예측 평점(est)데이터를 튜플 형태로 가지고 있다. 

* was_impossible이 true이면 예측값을 생성할 수 없는 데이터라는 의미이다.

In [8]:
#3개의 prediction객체에서 uid,iid,est속성을 추출
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.6682630479069185),
 ('882', '291', 3.8825855670016187),
 ('535', '507', 4.065480493123735)]

### predict()를 이용해 추천 예측

- 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환한다.

In [9]:
#사용자 아이디, 아이템 아이디는 문자열로 입력해야함.
uid=str(196)
iid=str(302)
pred=algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.17   {'was_impossible': False}


### 추천 예측 평점과 실제 평점과의 차이 평가

In [10]:
accuracy.rmse(predictions)

RMSE: 0.9458


0.945771026253141



---
### Surprise 주요 모듈 소개

### Dataset

* 반드시 데이터 세트의 칼럼 순서가 사용자 아이디, 아이템 아이디, 평점 순

1. Dataset.load_builtin(name='ml-100k')

무비렌즈 아카이브 FTP서버에서 무비 렌즈 데이터를 내려 받는다.

2. Dataset.load_from_file(file_path, reader)

OS파일에서 데이터를 로딩할 때 사용

3. Dataset.load_from_df

판다스의 DataFrame에서 데이터를 로디

### OS파일 데이터를 surprise데이터 세트로 로딩

1. 지정된 디렉토리에 있는 사용자-아이템 평점 데이터를 로딩
* 주의점 : 로딩 되는 데이터 파일에 칼럼명을 가지는 헤더 문자열에 있어서는 안 됨.(삭제하고 진행)

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
import pandas as pd

ratings=pd.read_csv('/content/drive/MyDrive/ESAA_data/ratings.csv')
#ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('/content/drive/MyDrive/ESAA_data/ratings_noh.csv', index=False, header=False)

다음으로 Reader 클래스를 이용해 데이터 파일의 파싱 포맷을 정의한다.

이 4개의 칼럼이 사용자 아이디, 아이템 아이디, 평점, 타임스탬프임을 로딩할 때 알려줘야한다.

In [25]:
#해당 설정된 것을 기반으로 데이터를 파싱하며 로딩함.
from surprise import Reader

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/drive/MyDrive/ESAA_data/ratings_noh.csv', reader=reader)

* line_format : 칼럼을 순서대로 나열
* sep : 칼럼을 분리하는 분리자
* rating_scale : 평점 값의 최소~최대 평점을 설정

### SVD 행렬 분해 기법을 이용해 추천 예측


In [26]:
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

#수행시마다 동일한 결과를 도출하기 위해 random_state 설정
algo=SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 테스트 데이터 세트로 평점 예측 후 rmse 평가
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 판다스 Dataframe에서 surprise데이터 세트로 로딩



In [28]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv('/content/drive/MyDrive/ESAA_data/ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))

#ratings df에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Surprise 추천 알고리즘 클래스

1. SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD알고리즘
2. KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN알고리즘
3. BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD베이스라인 알고리즘

-------
1. SVD

n_factors : 잠재 요인 k의 개수

n_epochs: SGD수행시 반복 횟수. 디폴트 20

biased : 베이스라인 사용자 편향 적용 여부. 디폴트는 True



---

### 베이스라인 평점

각 개인의 성향에 따라 같은 아이템이더라도 평가가 달라질 수 있다. 이러한 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것을 베이스라인 평점이라고 한다.

> 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수

* 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
* 사용자 편향 점수 = 사용자별 ㅏㅇ이템 평점 평균 값 - 전체 평균 평점
* 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균평점

### 교차 검증과 하이퍼 파라미터 튜닝

> cross_validate() 와 gridSearchCV클래스 제공함.

In [31]:
from surprise.model_selection import cross_validate

#판다스 df에서 surprise 데이터 세트로 데이터 로딩
ratings=pd.read_csv('/content/drive/MyDrive/ESAA_data/ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo=SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
     

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8664  0.8850  0.8809  0.8728  0.8685  0.8747  0.0071  
MAE (testset)     0.6640  0.6784  0.6782  0.6725  0.6672  0.6721  0.0058  
Fit time          3.82    1.68    1.64    1.66    1.65    2.09    0.86    
Test time         0.25    0.14    0.14    0.15    0.15    0.16    0.04    


{'test_rmse': array([0.86640293, 0.88496734, 0.88087637, 0.87277004, 0.86851531]),
 'test_mae': array([0.66403461, 0.67842067, 0.67816232, 0.67250298, 0.66724627]),
 'fit_time': (3.817003011703491,
  1.681147575378418,
  1.6393404006958008,
  1.6640527248382568,
  1.6476421356201172),
 'test_time': (0.24950671195983887,
  0.137192964553833,
  0.1407020092010498,
  0.14642786979675293,
  0.14526963233947754)}

-> 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치

### 파라미터 튜닝 진행

In [32]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid={'n_epochs':[20, 40, 60],
            'n_factors':[50, 100, 200]}

#CV를 3개 폴드 세트로 지정, 성능 평가를 rmse, mse로 수행하도록 gridsearchcv 구성
gs=GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

#최고 RMSE evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8769331573617863
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현

이제 surprise패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천해보자.

In [ ]:
# 분리하지 않고 데이터 사용시 오류

#data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#algo=SVD(n_factors=50, random_state=0)
#algo.fit(data)

데이터 세트 전체를 학습 데이터로 사용하려면 ? DatasetAutoFolds 클래스를 사용!

In [39]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
#datasetaufofolds 클래스를 ratings_noh.csv 기반으로 생성
data_folds=DatasetAutoFolds(ratings_file='/content/drive/MyDrive/ESAA_data/ratings_noh.csv', reader=reader)

#전체 데이터를 학습 데이터로 생성
trainset=data_folds.build_full_trainset()

In [34]:
algo=SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [41]:
#특정 사용자 9로 지정하기.
#영화에 대한 상세 속성 정보 DataFrame 로딩
movies=pd.read_csv('/content/drive/MyDrive/ESAA_data/movies.csv')

#userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds=ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [42]:
uid=str(9)
iid=str(42)

pred=algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


이제 사용자가 평점을 매기지 않은 전체 영화를 추출하고, 예측 평점 순으로 영화를 추천해보자. 

In [43]:
def get_unseen_surprise(ratings, movies, userId):
  #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()

  #모든 영화의 movieId를 리스트로 생성
  total_movies=movies['movieId'].tolist()

  #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies=get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


함수 생성) 인자로 학습이 완료된 추천 알고리즘 객체, 추천 대상 사용자 아이디, 추천 대상 영화의 리스트 객체, 그리고 추천 상위 N개 개수를 받는다.

In [44]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list로 저장
  predictions=[algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  #predictions list 객체는 surprise의 predictions 객체를 원소로 갖고 있다.
  # [Prediction(uid='9', iid='1', est=3.69), Predictions(uid='9', iid='2', est=2.98),,,]

  #이를 est 값으로 정렬하기 위해 아리애 sortkey_est 함수 정의
  #sortdey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  #sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions=predictions[:top_n]

  #top_n 으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids=[int(pred.iid) for pred in top_predictions]
  top_movie_rating=[pred.est for pred in top_predictions]
  top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']
  top_movie_preds=[ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies=get_unseen_surprise(ratings, movies, 9)
top_movie_preds=recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('### top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
### top-10 추천 영화 리스트 ###
Toy Story (1995) : 3.5002644558160445
Jumanji (1995) : 3.5002644558160445
Grumpier Old Men (1995) : 3.5002644558160445
Waiting to Exhale (1995) : 3.5002644558160445
Father of the Bride Part II (1995) : 3.5002644558160445
Heat (1995) : 3.5002644558160445
Sabrina (1995) : 3.5002644558160445
Tom and Huck (1995) : 3.5002644558160445
Sudden Death (1995) : 3.5002644558160445
GoldenEye (1995) : 3.5002644558160445
